In [1]:
import pandas as pd
import re


In [2]:
input_file_path = '/Users/thebekhruz/Desktop/100Days-Of-Code/100-Days-of-NLP-Odyssey/data/intermediate/swop_triples_cleaned.csv'
df = pd.read_csv(input_file_path, delimiter='\t', header=None, names=['doc_id', 'type', 'value'])
df.head()


,doc_id,type,value
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,title,View towards SE of junction of Queen Victoria ...
1,a7bb9917-95ff-3f55-a640-4c5afcec25f2,description,E corner of Queen Victoria Rd at junction with...
2,a7bb9917-95ff-3f55-a640-4c5afcec25f2,mentions,https://www.wikidata.org/wiki/Q64116
3,a7bb9917-95ff-3f55-a640-4c5afcec25f2,mentions,https://www.wikidata.org/wiki/Q64116
4,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,title,"Front page of Bucks Free Press, Time capsule f..."


In [3]:
title_df = df[df['type'] == 'title']['value'].to_frame(name='title')
title_df.head()

,title
1,E corner of Queen Victoria Rd at junction with...
5,Front page Bucks Free Press The Time Capsule T...
9,New Police Station High Wycombe viewed from op...
12,Corridor entrance to Reference Library
16,Terrace of brick and flint cottages


> Expand abbrivations that you have in the dictionary and then check for those which you dont have.

In [4]:
ABBREVIATION_EXPANSIONS = {
    # Geographical:
    "PH": "Public house",
    "R.Wye": "River Wye",
    "HW": "High Wycombe",
    "HQ": "Headquarters",
    "HDC": "High Wycombe District Council",
    # Historical and Military:
    "W.R.A.F.": "Women's Royal Air Force.",
    "V.E.Day": "Victory in Europe Day", 'VE Day': "Victory in Europe Day",
    "WWI": "First World War",
    "WWII": "Second World War",
    "RAF": "Royal Air Force",
    "HMS": "Her Majesty's Ship",
    "HRH": "His Royal Highness",
    "MP": "Member of Parliament",
    "BFP": "Bucks Free Press",
    "VJ Day": "Victory over Japan Day", "VJ-day": "Victory over Japan Day",
    # Compass directions
    "N": "North",
    "S": "South",
    "E": "East",
    "W": "West",
    "NE": "Northeast",
    "NW": "Northwest",
    "SE": "Southeast",
    "SW": "Southwest",
    "NNE": "North-Northeast",
    "ENE": "East-Northeast",
    "ESE": "East-Southeast",
    "SSE": "South-Southeast",
    "SSW": "South-Southwest",
    "WSW": "West-Southwest",
    "WNW": "West-Northwest",
    "NNW": "North-Northwest",
    # Street Names
    "St": "Street",
    "Rd": "Road",
    "Ave": "Avenue",
    "Dr": "Drive",
    "Ln": "Lane",
    "Ct": "Court",
    "Sq": "Square",
    # Dates
    "c.": "circa", "c" : "circa",
    "Mon.": "Monday ", "Mon": "Monday",
    "Tue.": "Tuesday ", "Tue": "Tuesday",
    "Wed.": "Wednesday ", "Wed": "Wednesday",
    "Thu.": "Thursday ", "Thu": "Thursday",
    "Fri.": "Friday ", "Fri": "Friday",
    "Sat.": "Saturday ", "Sat": "Saturday",
    "Sun.": "Sunday ", "Sun": "Sunday",
    "Jan.": "January ", "Jan": "January ",
    "Feb.": "February ", "Feb": "February",
    "Mar.": "March ", "Mar": "March",
    "Apr.": "April ", "Apr": "April",
    "May": "May",
    "Jun.": "June ", "Jun": "June",
    "Jul.": "July ", "Jul": "July",
    "Aug.": "August ", "Aug": "August",
    "Sep.": "September ", "Sep": "September ", "Sept.": "September ", "Sept": "September ",
    "Oct.": "October ", "Oct": "October",
    "Nov.": "November ", "Nov": "November",
    "Dec.": "December ", "Dec": "December",
    # Other
    "FC": "Football Club"

    # Extend this list as needed
}

In [5]:
def expand_abbreviations(text):
    """Replace abbreviations in the text with their expanded forms."""
    for abbr, expansion in ABBREVIATION_EXPANSIONS.items():
        pattern = r'\b' + re.escape(abbr) + r'\b(?![\'’])'
        text = re.sub(pattern, expansion, text, flags=re.IGNORECASE)
    return text

title_df['expanded_title'] = title_df['title'].astype(str).apply(expand_abbreviations)
title_df.head()

,title,expanded_title
1,E corner of Queen Victoria Rd at junction with...,East corner of Queen Victoria Road at junction...
5,Front page Bucks Free Press The Time Capsule T...,Front page Bucks Free Press The Time Capsule T...
9,New Police Station High Wycombe viewed from op...,New Police Station High Wycombe viewed from op...
12,Corridor entrance to Reference Library,Corridor entrance to Reference Library
16,Terrace of brick and flint cottages,Terrace of brick and flint cottages


In [6]:
non_empty_abbreviations = title_df[title_df['expanded_title'].apply(lambda x: len(x) > 0)]
non_empty_abbreviations.head()

,title,expanded_title
1,E corner of Queen Victoria Rd at junction with...,East corner of Queen Victoria Road at junction...
5,Front page Bucks Free Press The Time Capsule T...,Front page Bucks Free Press The Time Capsule T...
9,New Police Station High Wycombe viewed from op...,New Police Station High Wycombe viewed from op...
12,Corridor entrance to Reference Library,Corridor entrance to Reference Library
16,Terrace of brick and flint cottages,Terrace of brick and flint cottages


In [7]:
pattern = r'\b[B-Z]{2,3}\b'
title_df['abbreviations'] = title_df['expanded_title'].astype(str).apply(lambda x: re.findall(pattern, x))


In [8]:
# List of common Roman numerals to exclude
roman_numerals = {'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'L', 'C', 'D', 'M'}

# Function to filter out Roman numerals from the abbreviations list
def filter_roman_numerals(abbreviations):
    return [abbr for abbr in abbreviations if abbr not in roman_numerals]

# Apply the filter function to each row's 'abbreviations' column
title_df['abbreviations'] = title_df['abbreviations'].apply(filter_roman_numerals)


In [9]:
# Step 1: Flatten the list of abbreviations and count occurrences
abbreviation_counts = pd.Series([abbr for sublist in title_df['abbreviations'] for abbr in sublist]).value_counts()

# Step 2: Filter abbreviations that appear more than 10 times
frequent_abbreviations = abbreviation_counts[abbreviation_counts > 20].index.tolist()

# Step 3: Define a function to check if a row contains any of the frequent abbreviations
def contains_frequent_abbreviation(abbreviations):
    return any(abbr in frequent_abbreviations for abbr in abbreviations)

# Step 4: Apply this function to filter rows in the original DataFrame
filtered_df = title_df[title_df['abbreviations'].apply(contains_frequent_abbreviation)]
# filtered_df = title_df[title_df['abbreviations'].apply(lambda x: len(x) > 1)]

# Display the filtered DataFrame
with pd.option_context('display.max_rows', None):
    if not filtered_df.empty:
        display(filtered_df)

,title,expanded_title,abbreviations
23,"Row of cottages, wash houses and part of facto...","Row of cottages, wash houses and part of facto...",[HDC]
173,"Celebration of retirement, showing commemorati...","Celebration of retirement, showing commemorati...",[UK]
248,"Demolition of house in Mellet's Yard, and the ...","Demolition of house in Mellet'South Yard, and ...",[HDC]
510,"Row of terraced cottages on N side of Ford St,...",Row of terraced cottages on North side of Ford...,[HDC]
549,Demolition of houses in Brook St showing laund...,Demolition of houses in Brook Street showing l...,[HDC]
568,Cottages with timber outhouse. these are the p...,Cottages with timber outhouse. these are the p...,[HDC]
588,looking Eastward from the upper floor of cotta...,looking Eastward from the upper floor of cotta...,[HDC]
997,Hairdressing Permanent and Marcel Waves (Price...,Hairdressing Permanent and Marcel Waves (Price...,[TV]
1164,"Front of Royal Grammar School in Easton St, bu...",Front of Royal Grammar School in Easton Street...,[HDC]
2156,View of front of Royal Grammar School Amersham...,View of front of Royal Grammar School Amersham...,[HDC]


In [10]:
output_file_path = '/Users/thebekhruz/Desktop/100Days-Of-Code/100-Days-of-NLP-Odyssey/data/intermediate/preprocessing/abbrivations.csv'
df.to_csv(output_file_path, sep='\t', index=False, header=None)
